Same thing as other notebook except conv net was constructed from grey images. As you can see, basic neural nets overfitted tremendously so the conv net was a massive improvement.

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports for our work
import os 
import numpy as np
import tensorflow as tf
import time
import pandas as pd

import tensorflow as tf
import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.metrics import Accuracy

RANDOM_SEED = 9999

# To make output stable across runs
def reset_graph(seed= RANDOM_SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

### Uploading and formatting data

In [2]:
# CatsDogs:
# Documentation on npy binary format for saving numpy arrays for later use
#     https://towardsdatascience.com/
#             why-you-should-start-using-npy-file-more-often-df2a13cc0161
# Under the working directory, data files are in directory cats_dogs_64_128 
# Read in cats and dogs grayscale 64x64 files to create training data

# 1000 dog and 1000 cat images
# greyscale: each image is a 128x128x1 pixel array (the last dim is the color)
cats_1000_128_128_1 = np.load('cats_dogs_images/greyscale/cats_1000_128_128_1.npy')
dogs_1000_128_128_1 = np.load('cats_dogs_images/greyscale/dogs_1000_128_128_1.npy')

# color: each image is a 64x64x3 pixel array (the "x3" is the color for RGB dimensions)
cats_1000_64_64_3 = np.load('cats_dogs_images/color/cats_1000_64_64_3.npy')
dogs_1000_64_64_3 = np.load('cats_dogs_images/color/dogs_1000_64_64_3.npy')


In [3]:
from matplotlib import pyplot as plt  # for display of images
def show_grayscale_image(image):
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()
    
def show_color_image(image):
    plt.imshow(image) 
    plt.axis('off')
    plt.show()   

# Examine first cat and first dog grayscale images
show_grayscale_image(cats_1000_128_128_1[0,:,:,0])
show_grayscale_image(dogs_1000_128_128_1[0,:,:,0])

show_color_image(cats_1000_64_64_3[0,:,:,0])
show_color_image(dogs_1000_64_64_3[0,:,:,0])

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [4]:
# Work the data for cats and dogs numpy arrays 
# These numpy arrays were generated in previous data prep work
# Stack the numpy arrays for the inputs

X_cat_dog_grey = np.concatenate((cats_1000_128_128_1, dogs_1000_128_128_1), axis = 0)

X_cat_dog_color = np.concatenate((cats_1000_64_64_3, dogs_1000_64_64_3), axis = 0)

In [5]:
print(X_cat_dog_color.shape)
print(X_cat_dog_grey.shape)

(2000, 64, 64, 3)
(2000, 128, 128, 1)


In [6]:
# to get an array of 2000 arrays of length 16384
X_cat_dog_grey = X_cat_dog_grey.reshape(-1, 128*128)

##### Min-max:

In [7]:
# Scikit Learn for min-max scaling of the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(np.array([0., 255.]).reshape(-1,1))
X_cat_dog_grey_min_max = scaler.transform(X_cat_dog_grey)


In [8]:
X_cat_dog_grey_min_max[0].shape

(16384,)

##### Robust:

In [9]:
#Scikit Learn for Robust Scaling scaling of the data
from sklearn.preprocessing import RobustScaler
scaler2 = RobustScaler()
scaler2.fit(np.array([0., 255.]).reshape(-1,1)) 
X_cat_dog_grey_robust = scaler2.transform(X_cat_dog_grey.reshape(-1,128*128))

In [10]:
X_cat_dog_grey_robust[0]

array([ 0.30980392,  0.35686275,  0.36470588, ..., -0.98431373,
       -0.98431373, -0.98431373])

In [11]:
# Define the labels to be used 1000 cats = 0 1000 dogs = 1
y_cat_dog = np.concatenate((np.zeros((1000), dtype = np.int32), 
                      np.ones((1000), dtype = np.int32)), axis = 0)

In [12]:
# Scikit Learn for random splitting of the data  
from sklearn.model_selection import train_test_split

##### Splitting into training sets by min-max and robust scaling

In [13]:
# Min-max Split
# training (80%) and test (20%)  
X_train_min_max, X_test_min_max, y_train_min_max, y_test_min_max = \
    train_test_split(X_cat_dog_grey_min_max, y_cat_dog, test_size=0.20, 
                     random_state = RANDOM_SEED)

In [14]:
# Robust Split
# training (80%) and test (20%)  
X_train_r, X_test_r, y_train_r, y_test_r = \
    train_test_split(X_cat_dog_grey_robust, y_cat_dog, test_size=0.20, 
                     random_state = RANDOM_SEED)

In [15]:
# Normal train-test split and reshaping for convolutional neural net at the end
X_train, X_test, y_train, y_test = \
    train_test_split(X_cat_dog_grey, y_cat_dog, test_size=0.20, 
                     random_state = RANDOM_SEED)

In [16]:
X_train = X_train.reshape(-1, 128, 128, 1)
X_test = X_test.reshape(-1, 128, 128, 1)

# leaves 2 options that the conv net can choose from
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [17]:
X_train.shape

(1600, 128, 128, 1)

## Setting up tflow models

In [18]:
# setup - define neuron layer
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [19]:
#set height & width
height = 128
width = 128 

# 300 and 100 nodes for layers 1 and 2 as used with MNIST from Geron
n_hidden1 = 300
n_hidden2 = 100


channels = 1  # When working with color images use channels = 3

n_inputs = height * width

#CatsDogs# Has two output values # MNIST had ten digits n_outputs = 10  
n_outputs = 2  # binary classification for Cats and Dogs, 1 output node 0/1

reset_graph()

# dnn... Deep neural network model from Geron Chapter 10
# Note that this model makes no use of the fact that we have
# pixel data arranged in rows and columns
# So a 64x64 matrix of raster values becomes a vector of 4096 input variables
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [20]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    

init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 50 epochs using min-max scaled data

In [21]:
%%time

init = tf.global_variables_initializer()    

n_epochs = 50
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(y_train_min_max.shape[0] // batch_size):
            X_batch_min_max = X_train_min_max[iteration*batch_size:(iteration + 1)*batch_size,:]
            y_batch_min_max = y_train_min_max[iteration*batch_size:(iteration + 1)*batch_size]
            sess.run(training_op, feed_dict={X: X_batch_min_max, y: y_batch_min_max})
        acc_train = accuracy.eval(feed_dict={X: X_batch_min_max, y: y_batch_min_max})
        acc_test = accuracy.eval(feed_dict={X: X_test_min_max, y: y_test_min_max})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./my_catdog_model_min_max")

0 Train accuracy: 0.61 Test accuracy: 0.485
1 Train accuracy: 0.6 Test accuracy: 0.485
2 Train accuracy: 0.61 Test accuracy: 0.485
3 Train accuracy: 0.62 Test accuracy: 0.495
4 Train accuracy: 0.62 Test accuracy: 0.4925
5 Train accuracy: 0.63 Test accuracy: 0.5
6 Train accuracy: 0.65 Test accuracy: 0.5025
7 Train accuracy: 0.68 Test accuracy: 0.505
8 Train accuracy: 0.67 Test accuracy: 0.5175
9 Train accuracy: 0.68 Test accuracy: 0.5125
10 Train accuracy: 0.7 Test accuracy: 0.5325
11 Train accuracy: 0.72 Test accuracy: 0.5325
12 Train accuracy: 0.69 Test accuracy: 0.5275
13 Train accuracy: 0.69 Test accuracy: 0.5225
14 Train accuracy: 0.69 Test accuracy: 0.525
15 Train accuracy: 0.7 Test accuracy: 0.54
16 Train accuracy: 0.74 Test accuracy: 0.55
17 Train accuracy: 0.69 Test accuracy: 0.535
18 Train accuracy: 0.73 Test accuracy: 0.53
19 Train accuracy: 0.66 Test accuracy: 0.525
20 Train accuracy: 0.77 Test accuracy: 0.545
21 Train accuracy: 0.71 Test accuracy: 0.54
22 Train accuracy: 0.

# 50 epochs using robust scaled data

In [22]:
%%time
init = tf.global_variables_initializer()  

startTime = time.time()
n_epochs = 50
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(y_train_r.shape[0] // batch_size):
            X_batch_r = X_train_r[iteration*batch_size:(iteration + 1)*batch_size,:]
            y_batch_r = y_train_r[iteration*batch_size:(iteration + 1)*batch_size]
            sess.run(training_op, feed_dict={X: X_batch_r, y: y_batch_r})
        acc_train_3 = accuracy.eval(feed_dict={X: X_batch_r, y: y_batch_r})
        acc_test_3 = accuracy.eval(feed_dict={X: X_test_r, y: y_test_r})
        print(epoch, "Train accuracy:", acc_train_3, "Test accuracy:", acc_test_3)

        save_path = saver.save(sess, "./my_catdog_model_robust")

0 Train accuracy: 0.79 Test accuracy: 0.505
1 Train accuracy: 0.91 Test accuracy: 0.51
2 Train accuracy: 0.92 Test accuracy: 0.525
3 Train accuracy: 0.94 Test accuracy: 0.5375
4 Train accuracy: 0.95 Test accuracy: 0.5375
5 Train accuracy: 0.97 Test accuracy: 0.5425
6 Train accuracy: 0.97 Test accuracy: 0.5425
7 Train accuracy: 0.97 Test accuracy: 0.55
8 Train accuracy: 1.0 Test accuracy: 0.55
9 Train accuracy: 1.0 Test accuracy: 0.5575
10 Train accuracy: 1.0 Test accuracy: 0.5525
11 Train accuracy: 1.0 Test accuracy: 0.5575
12 Train accuracy: 1.0 Test accuracy: 0.5625
13 Train accuracy: 1.0 Test accuracy: 0.5575
14 Train accuracy: 1.0 Test accuracy: 0.5625
15 Train accuracy: 1.0 Test accuracy: 0.5625
16 Train accuracy: 1.0 Test accuracy: 0.56
17 Train accuracy: 1.0 Test accuracy: 0.56
18 Train accuracy: 1.0 Test accuracy: 0.56
19 Train accuracy: 1.0 Test accuracy: 0.5575
20 Train accuracy: 1.0 Test accuracy: 0.5525
21 Train accuracy: 1.0 Test accuracy: 0.5475
22 Train accuracy: 1.0 Tes

In [23]:
## input
input_layer = input_data(shape=[None, 128, 128, 1], name='input') # dimensions of each of the imgs

## middle layers
conv1 = conv_2d(input_layer, 64, 4, activation='relu')
net = max_pool_2d(conv1, 5)

conv2 = conv_2d(net, 128, 2, activation='relu')
net = max_pool_2d(conv2, 5)

conv3 = conv_2d(net, 128, 1, activation='relu')
net = dropout(conv3, 0.5)

conv_4 = conv_2d(net, 128, 1, activation='relu', name='conv_3')
net = max_pool_2d(conv_4, 2)

# Fully-connected 512 node layer
network = fully_connected(net, 384, activation='relu')
# Dropout layer to combat overfitting
network = dropout(network, 0.5)

## output layer
output = fully_connected(network, 2, activation='softmax')
acc = Accuracy(name='Accuracy')
output = regression(output,
                    optimizer='adam',
                    learning_rate=0.0005, 
                    loss='categorical_crossentropy',
                    metric=acc)

model = tflearn.DNN(output, checkpoint_path='conv_grey_net.tflearn', max_checkpoints = 3,
                    tensorboard_verbose = 3, tensorboard_dir='tmp/tflearn_logs/')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [24]:
%%time
model.fit(X_train, y_train, validation_set=(X_test, y_test), batch_size = 400, n_epoch = 100, show_metric=True)
              #,verbose = 0, shuffle=True, 
              #callbacks=[early_stopping])

Training Step: 399  | total loss: 0.34927 | time: 44.649s
| Adam | epoch: 100 | loss: 0.34927 - Accuracy: 0.8470 -- iter: 1200/1600
Training Step: 400  | total loss: 0.34983 | time: 60.809s
| Adam | epoch: 100 | loss: 0.34983 - Accuracy: 0.8458 | val_loss: 0.62666 - val_acc: 0.6775 -- iter: 1600/1600
--
CPU times: user 5h 11min 24s, sys: 34min 59s, total: 5h 46min 23s
Wall time: 1h 46min 28s


##### Saving model

In [25]:
# https://github.com/tflearn/tflearn/blob/master/examples/basics/weights_persistence.py
model.save('tf_cat_dog_grey_conv_net.tfl')  # manually creates a tflearn file
